In [23]:
# Import necessary libraries
import numpy as np
import nibabel as nib
from ipywidgets import interact, IntSlider, Button, HBox, VBox, Dropdown
import matplotlib.pyplot as plt
import glob

# Initialize global variables
current_image_data = None
current_label_data = None

# Function to load image and get slice thickness
def load_image(image_path):
    image_obj = nib.load(image_path)
    return image_obj.get_fdata(), np.abs(image_obj.affine[2, 2])

# Function to explore single image and label
def explore_image_and_label(layer, channel):
    global current_image_data, current_label_data, current_slice_thickness
    
    if current_image_data is None or current_label_data is None:
        return
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(current_image_data[:, :, layer, channel], cmap='gray')
    plt.title(f'MRI Image\nSlice Thickness: {current_slice_thickness} mm')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    img = plt.imshow(current_label_data[:, :, layer], cmap='jet', alpha=0.5)
    plt.title('Label Overlay')
    plt.axis('off')
    
    cbar = plt.colorbar(img, orientation='vertical', ticks=[0, 1, 2, 3])
    cbar.set_label('Class Labels')
    cbar.set_ticklabels(['Normal', 'Edema', 'Non-enhancing tumor', 'Enhancing tumor'])
    
    plt.tight_layout()
    plt.show()

# Function to update current image and label data
def update_data(file_index):
    global current_image_data, current_label_data, current_slice_thickness
    
    if file_index is None:
        return
    
    image_files = sorted(glob.glob("./data/images/*.nii.gz"))
    label_files = sorted(glob.glob("./data/labels/*.nii.gz"))
    
    current_image_data, current_slice_thickness = load_image(image_files[file_index])
    current_label_data, _ = load_image(label_files[file_index])
    layer_slider.max = current_image_data.shape[2] - 1
    channel_slider.max = current_image_data.shape[3] - 1

# Create widgets and layout
file_options = [(f"File {i+1}", i) for i in range(len(glob.glob("./data/images/*.nii.gz")))]
file_selector = Dropdown(options=file_options, description="Select File:")
layer_slider = IntSlider(min=0, max=100, description="Layer:")
channel_slider = IntSlider(min=0, max=3, description="Channel:")

file_box = HBox([file_selector])
layer_box = HBox([layer_slider])
channel_box = HBox([channel_slider])
ui = VBox([file_box, layer_box, channel_box])

# Display UI
_ = interact(explore_image_and_label, layer=layer_slider, channel=channel_slider)
_ = interact(update_data, file_index=file_selector)


interactive(children=(IntSlider(value=0, description='Layer:'), IntSlider(value=0, description='Channel:', max…

interactive(children=(Dropdown(description='Select File:', options=(('File 1', 0), ('File 2', 1), ('File 3', 2…